A loadmat function for when you get the following error using `scipy.io.loadmat`:
```
---------------------------------------------------------------------------
NotImplementedError                       Traceback (most recent call last)
<ipython-input-16-200704b4a286> in <module>
      1 from scipy.io import loadmat
----> 2 loadmat(FILE)

~/miniconda3/envs/sci/lib/python3.6/site-packages/scipy/io/matlab/mio.py in loadmat(file_name, mdict, appendmat, **kwargs)
    215     variable_names = kwargs.pop('variable_names', None)
    216     with _open_file_context(file_name, appendmat) as f:
--> 217         MR, _ = mat_reader_factory(f, **kwargs)
    218         matfile_dict = MR.get_variables(variable_names)
    219 

~/miniconda3/envs/sci/lib/python3.6/site-packages/scipy/io/matlab/mio.py in mat_reader_factory(file_name, appendmat, **kwargs)
     76         return MatFile5Reader(byte_stream, **kwargs), file_opened
     77     elif mjv == 2:
---> 78         raise NotImplementedError('Please use HDF reader for matlab v7.3 files')
     79     else:
     80         raise TypeError('Did not recognize version %s' % mjv)

NotImplementedError: Please use HDF reader for matlab v7.3 files
```

In [1]:
import h5py
import numpy as np

Dataset = h5py._hl.dataset.Dataset
Group = h5py._hl.group.Group
Reference = h5py.h5r.Reference


def loadmat_h5(file_name):
    '''Loadmat equivalent for -v7.3 or greater .mat files, which break scipy.io.loadmat'''
    def deref_s(s, f, verbose=False):  # dereference struct
        keys = [k for k in s.keys() if k != '#refs#']

        if verbose:
            print(f'\nStruct, keys = {keys}')

        d = {}

        for k in keys:
            v = s[k]

            if isinstance(v, Group):  # struct
                d[k] = deref_s(v, f, verbose=verbose)

            elif isinstance(v, Dataset) and isinstance(np.array(v).flat[0], Reference):  # cell
                d[k] = deref_c(v, f, verbose=verbose)

            elif isinstance(v, Dataset) and v.dtype == 'uint16':
                d[k] = ''.join(np.array(v).view('S2').flatten().astype(str))

                if verbose:
                    print(f'String, chars = {d[k]}')

            elif isinstance(v, Dataset):  # numerical array
                d[k] = np.array(v).T

                if verbose:
                    print(f'Numerical array, shape = {d[k].shape}')

        return d


    def deref_c(c, f, verbose=False):  # dereference cell
        n_v = c.size
        shape = c.shape

        if verbose:
            print(f'\nCell, shape = {shape}')

        a = np.zeros(n_v, dtype='O')

        for i in range(n_v):
            v = f['#refs#'][np.array(c).flat[i]]

            if isinstance(v, Group):  # struct
                a[i] = deref_s(v, f, verbose=verbose)

            elif isinstance(v, Dataset) and isinstance(np.array(v).flat[0], Reference):  # cell
                a[i] = deref_c(v, f, verbose=verbose)

            elif isinstance(v, Dataset) and v.dtype == 'uint16':
                a[i] = ''.join(np.array(v).view('S2').flatten().astype(str))

                if verbose:
                    print(f'String, chars = {a[i]}')

            elif isinstance(v, Dataset):  # numerical array
                a[i] = np.array(v).T

                if verbose:
                    print(f'Numerical array, shape = {a[i].shape}')

        return a.reshape(shape).T
    
    with h5py.File(file_name, 'r+') as f:
        d = deref_s(f, f)
        
    return d

Usage

In [2]:
d = loadmat_h5('test_data.mat')

In [3]:
d.keys()

dict_keys(['data', 'experiment_class', 'experiment_id'])

In [4]:
d['data'].keys()

dict_keys(['metadata', 'my3tensor', 'my4tensor', 'mycol', 'mymatrix', 'myrow', 'stim_seq', 'timestamp', 'trials'])

In [5]:
d['data']['metadata']

{'date': '20201106', 'n_trial': array([[3.]]), 'protocol': 'natural_stim'}

In [6]:
d['data']['trials'].shape

(1, 3)

In [7]:
d['data']['trials'][0, 0].shape

(100, 4)

In [8]:
d['data']['trials'][0, 1].shape

(100, 4)

In [9]:
d['data']['trials'][0, 2].shape

(300, 4)

In [10]:
d['data']['mycol'].shape

(10, 1)

In [11]:
d['data']['myrow'].shape

(1, 10)

In [12]:
d['data']['mymatrix'].shape

(2, 4)

In [13]:
d['data']['my3tensor'].shape

(3, 5, 7)

In [14]:
d['data']['my4tensor'].shape

(2, 4, 6, 8)